# Ajuste de Modelos

In [241]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
import pandas as pd

In [242]:
# warnings are ignored to avoid cluttering the output
import warnings
warnings.filterwarnings("ignore")

In [243]:
url = "https://vincentarelbundock.github.io/Rdatasets/csv/AER/RecreationDemand.csv"
df = pd.read_csv(url)
df = df.drop(columns=["rownames"])

In [244]:
df['ski'] = df['ski'].astype('category')
df['userfee'] = df['userfee'].astype('category')

### Ajuste do Modelo de Poisson

In [245]:
poisson_model_complete = smf.glm(
    formula="trips ~ quality + income + ski + userfee + costC + costS + costH",
    data=df,
    family=sm.families.Poisson(),
).fit()

print(poisson_model_complete.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trips   No. Observations:                  659
Model:                            GLM   Df Residuals:                      651
Model Family:                 Poisson   Df Model:                            7
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1529.4
Date:                Sun, 25 May 2025   Deviance:                       2305.8
Time:                        21:03:35   Pearson chi2:                 4.10e+03
No. Iterations:                     8   Pseudo R-squ. (CS):             0.9789
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.2650      0.094      2.

In [246]:
poisson_model_simple = smf.glm(
    formula="trips ~ quality + income + ski + userfee + costS + costH",
    data=df,
    family=sm.families.Poisson(),
).fit()

print(poisson_model_simple.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trips   No. Observations:                  659
Model:                            GLM   Df Residuals:                      652
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1530.0
Date:                Sun, 25 May 2025   Deviance:                       2307.0
Time:                        21:03:35   Pearson chi2:                 4.08e+03
No. Iterations:                     7   Pseudo R-squ. (CS):             0.9789
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.2683      0.094      2.

### Teste de Sobredispersão

In [247]:
# Valores ajustados
mu_hat = poisson_model.fittedvalues
y = df["trips"]

# Estatistica Z_i
Zi = ((y - mu_hat)**2) - y / mu_hat

# Regressão de Z_i em mu_hat (sem intercepto)
sobredisp = sm.OLS(Zi, mu_hat).fit()
print(sobredisp.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.303
Model:                            OLS   Adj. R-squared (uncentered):              0.302
Method:                 Least Squares   F-statistic:                              286.4
Date:                Sun, 25 May 2025   Prob (F-statistic):                    1.32e-53
Time:                        21:03:35   Log-Likelihood:                         -4786.2
No. Observations:                 659   AIC:                                      9574.
Df Residuals:                     658   BIC:                                      9579.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [248]:
p_value = sobredisp.pvalues[0]
coef = sobredisp.params[0]

#print(f"P-value do teste de sobredispersão: {p_value}")
#print(f"Coeficiente estimado: {coef:.1f}")

# Teste de sobredispersão
if p_value < 0.05:
    print(f"O modelo apresenta sobredispersão, com p-value ({p_value}) < 0.05 e coeficiente estimado ({coef:.1f}) > 0.")
else:
    print("Não há evidências de sobredispersão nos dados.")

O modelo apresenta sobredispersão, com p-value (1.3248014712503913e-53) < 0.05 e coeficiente estimado (43.2) > 0.


## Modelos Alternativos

### Binomial Negativa

In [249]:
neg_bin_complete = smf.glm(
    formula="trips ~ quality + income + ski + userfee + costC + costS + costH",
    data=df,
    family=sm.families.NegativeBinomial(),
).fit()

print(neg_bin_complete.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trips   No. Observations:                  659
Model:                            GLM   Df Residuals:                      651
Model Family:        NegativeBinomial   Df Model:                            7
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -830.01
Date:                Sun, 25 May 2025   Deviance:                       503.97
Time:                        21:03:35   Pearson chi2:                 1.13e+03
No. Iterations:                    25   Pseudo R-squ. (CS):             0.7744
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0085      0.194     -5.

In [250]:
neg_bin_simple = smf.glm(
    formula = "trips ~ quality + ski + userfee + costC + costS + costH",
    data=df,
    family=sm.families.NegativeBinomial(),
).fit()

print(neg_bin_simple.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trips   No. Observations:                  659
Model:                            GLM   Df Residuals:                      652
Model Family:        NegativeBinomial   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -830.32
Date:                Sun, 25 May 2025   Deviance:                       504.59
Time:                        21:03:35   Pearson chi2:                 1.15e+03
No. Iterations:                    24   Pseudo R-squ. (CS):             0.7742
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1109      0.151     -7.

In [251]:
# compare all models

models_comparison = pd.DataFrame({
    'Model': ['Poisson Completo', 'Poisson Simples',
                'Neg Bin Completo', 'Neg Bin Simples'],
    'AIC': [m.aic for m in [
        poisson_model_complete, poisson_model_simple,
        neg_bin_complete, neg_bin_simple
    ]],
    'BIC': [m.bic for m in [
        poisson_model_complete, poisson_model_simple,
        neg_bin_complete, neg_bin_simple
    ]]
})

print(models_comparison.sort_values('AIC'))

              Model          AIC          BIC
3   Neg Bin Simples  1674.631200 -3727.366185
2  Neg Bin Completo  1676.012852 -3721.493810
1   Poisson Simples  3074.056775 -1924.972005
0  Poisson Completo  3074.862594 -1919.675463
